In [1]:
"""Development scripts for calculation beta
final version
"""
% matplotlib inline
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import datetime

import sys
sys.path.append('C:\\project\\code')
import setting

import prepare_data0301

In [2]:
data = prepare_data0301._load_raw_data('../input/stock300.csv')

daily_return, weekly_return, t = prepare_data0301.main_total_vol(data)

Start reading CSV...
Done reading CSV...
Total-Vol: Formatting data...


In [3]:
def _get_beta(daily_return):
    # prepare raw dataframe
    ff = pd.read_csv(setting.datapath+'famafrench.csv')
    ff['date'] = pd.to_datetime(ff['date'], format='%Y%m%d')
    ff.set_index('date', inplace=True)
    ff.columns = ['MKT_RF', 'SMB', 'HML', 'RF']
    daily_data = pd.concat(
        [daily_return, ff[['MKT_RF']]], 
        axis=1, 
        join='inner'
        )

    permno_list = [
            x for x in daily_data.columns
            if x not in ['MKT_RF']
            ]
    
    # convert log ret
    daily_log_ret = np.log(1.0+daily_data)
    overlap_log_ret = daily_log_ret.rolling(3).sum()
    
    # 1-year horizon for vol, daily log ret 
    mkt_vol = daily_log_ret['MKT_RF'].rolling(window= 251, min_periods=120).std()
    stk_vol = daily_log_ret[permno_list].rolling(window= 251, min_periods=120).std()
    
    # 5-year horizon for corr, sum of 3 daily log ret
    stk_corr = overlap_log_ret[permno_list].rolling(window=1255, min_periods=750).corr(overlap_log_ret['MKT_RF'])
    
    # calculation of beta
    # http://stackoverflow.com/questions/22642162/python-divide-each-row-of-a-dataframe-by-another-dataframe-vector
    stk_mkt = stk_vol.divide(mkt_vol, axis='index')
    stk_beta = stk_corr.multiply(stk_mkt)
    
    stk_beta_mthly = stk_beta.resample('M').last()
    stk_beta_mthly = stk_beta_mthly.to_period('M')
    
    stk_beta_mthly.stack(dropna=True).to_csv(setting.datapath_prepared+'beta.csv', header=['beta'])

In [4]:
_get_beta(daily_return)

In [43]:
# prepare raw dataframe
ff = pd.read_csv('../input/famafrench.csv')
ff['date'] = pd.to_datetime(ff['date'], format='%Y%m%d')
ff.set_index('date', inplace=True)
ff.columns = ['MKT_RF', 'SMB', 'HML', 'RF']
daily_data = pd.concat(
    [daily_return, ff[['MKT_RF']]], 
    axis=1, 
    join='inner'
    )

permno_list = [
        x for x in daily_data.columns
        if x not in ['MKT_RF']
        ]

In [ ]:
# convert log ret
daily_log_ret = np.log(1.0+daily_data)
overlap_log_ret = daily_log_ret.rolling(3).sum()

In [84]:
# 1-year horizon for vol, daily log ret 
mkt_vol = daily_log_ret['MKT_RF'].rolling(window= 251, min_periods=120).std()
stk_vol = daily_log_ret[permno_list].rolling(window= 251, min_periods=120).std()

# 5-year horizon for corr, sum of 3 daily log ret
stk_corr = overlap_log_ret[permno_list].rolling(window=1255, min_periods=750).corr(overlap_log_ret['MKT_RF'])

In [96]:
# calculation of beta
# http://stackoverflow.com/questions/22642162/python-divide-each-row-of-a-dataframe-by-another-dataframe-vector
stk_mkt = stk_vol.divide(mkt_vol, axis='index')
stk_beta = stk_corr.multiply(stk_mkt)

In [99]:
stk_beta_mthly = stk_beta.resample('M').last()
stk_beta_mthly = stk_beta_mthly.to_period('M')

In [101]:
stk_beta_mthly.stack(dropna=True).to_csv(setting.datapath_prepared+'beta.csv', header=['beta'])

In [81]:
import pandas as pd

data1 = {"a":[1.,3.,5.,2.],
         "b":[4.,8.,3.,7.],
         "c":[5.,45.,67.,34]}
data2 = [1, 1, 2, 2]

df1 = pd.DataFrame(data1)
df2 = pd.DataFrame(data2) 

df1.div(df2[0],axis='index')


,a,b,c
0,1.0,4.0,5.0
1,3.0,8.0,45.0
2,2.5,1.5,33.5
3,1.0,3.5,17.0
